In [1]:
%gui qt5
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor
import numpy as np

import theplotter.plugins.sources.hdf5 as hdf5
import theplotter.plugins.sources.spyview as spyview
from theplotter.plugins import transformations
from theplotter.plugins.targets import mpl
from theplotter.graph.nodes import SliceTransformation as slt
import theplotter.plugins.sources.generators as generators

pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


In [3]:
s1 = hdf5.HDF5Source('D:/data/2017-12-29/2017-12-29_086.h5', 'data')

In [69]:
s1.location = "D:/data/2018-01-08/2018-01-08_036.dat"
s1.need_update = True

In [56]:
s1 = spyview.SpyviewSource('D:/data/2018-01-08/2018-01-08_035.dat')

In [57]:
s1.play()

In [71]:
s1.dataset['Column 1'] = s1.dataset['Column 1']/10

In [61]:
tr1 = slt(s1, (0, slice(None), slice(None)))

In [89]:
slice_nr = 29
tr1.update_slice_operator((slice_nr, slice(None), slice(None)))

In [70]:
plotwindow = mpl.MPLWindow(nrows=2, ncols=2)
p1 = mpl.pcolormesh(datasources=[tr1['Column 1'], tr1['Column 2'], tr1['Column 4']], plottarget=plotwindow[0,0])
p2 = mpl.pcolormesh(datasources=[tr1['Column 1'], tr1['Column 2'], tr1['Column 13']], plottarget=plotwindow[0,1])
p3 = mpl.lineplot(datasources=[tr1[0, :]['Column 1'], transformations.basic.Average(0, datasources=[tr1['Column 4']])], plottarget=plotwindow[1,0])
p4 = mpl.lineplot(datasources=[tr1[0, :]['Column 1'], transformations.basic.Average(0, datasources=[tr1['Column 13']])], plottarget=plotwindow[1,1])


In [73]:
p1.add_widget('multicursor', MultiCursor, plotwindow.fig.canvas, (plotwindow[0,0].ax, plotwindow[0,1].ax), horizOn=True)

In [64]:
crop = transformations.basic.Crop2D(1, 0, 'Column 1', 'Column 2', datasources=[tr1])

In [85]:
crop.connect_plot(p1)

In [86]:
crop.connect_plot(p2)

In [14]:
std1 =  transformations.basic.STD(0, datasources=[crop['Column 13']])
std2 =  transformations.basic.STD(1, datasources=[crop['Column 13']])

In [67]:
std3 = transformations.basic.STD(0, datasources=[
        transformations.basic.Reshape((-1), datasources=[crop['Column 13']])
    ])

In [68]:
std3.play()
std3.get_data()

19.478685345183639

In [37]:
plotwindow2 = mpl.MPLWindow(nrows=2, ncols=2)

In [38]:

p5 = mpl.pcolormesh(datasources=[crop['Column 1'], crop['Column 2'], crop['Column 4']], plottarget=plotwindow2[0,0])
p6 = mpl.pcolormesh(datasources=[crop['Column 1'], crop['Column 2'], crop['Column 13']], plottarget=plotwindow2[0,1])
p7 = mpl.lineplot(datasources=[crop[0, :]['Column 1'], std1], plottarget=plotwindow2[1,0])
p8 = mpl.lineplot(datasources=[crop[:, 0]['Column 2'], std2], plottarget=plotwindow2[1,1])

In [49]:
p5.add_widget('multicursor', MultiCursor, plotwindow2.fig.canvas, (plotwindow2[0,0].ax, plotwindow2[0,1].ax), horizOn=True)

In [19]:
newgrid = generators.create_orthogonal_grid(100, 100)

In [39]:
newgrid.connect_plot(p6)

In [21]:
interpolated = transformations.basic.Interpolate(datasources=[tr1[['Column 1', 'Column 2']], tr1['Column 4'], newgrid])
interpolated2 = transformations.basic.Interpolate(datasources=[tr1[['Column 1', 'Column 2']], tr1['Column 13'], newgrid])

In [22]:
linspace = generators.from_array(np.array([[(i, j) for i in np.linspace(0,1,100)] for j in np.linspace(0, 100, 100)],dtype=np.dtype([('x',np.float64),('y',np.float64)])))

In [40]:
plotwindow3 = mpl.MPLWindow(nrows=4, ncols=2)
p9 = mpl.pcolormesh(datasources=[newgrid['x'], 
                                 newgrid['y'], 
                                 interpolated
                                ], plottarget=plotwindow3[0,0])
p10 = mpl.pcolormesh(datasources=[newgrid['x'], 
                                 newgrid['y'], 
                                 interpolated2
                                ], plottarget=plotwindow3[0,1])
p9b = mpl.pcolormesh(datasources=[linspace['x'], 
                                 linspace['y'], 
                                 interpolated
                                ], plottarget=plotwindow3[1,0])
p10b = mpl.pcolormesh(datasources=[linspace['x'], 
                                 linspace['y'], 
                                 interpolated2
                                ], plottarget=plotwindow3[1,1])

In [41]:
p11 = mpl.lineplot(datasources=[
        newgrid['x'][:,0]+newgrid['y'][:,0], 
        transformations.basic.Average(1, datasources=[interpolated])], 
        plottarget=plotwindow3[2,0])
p12 = mpl.lineplot(datasources=[
        newgrid['x'][:,0]+newgrid['y'][:,0], 
        transformations.basic.Average(1, datasources=[interpolated2])], 
        plottarget=plotwindow3[2,1])

In [42]:
p13 = mpl.lineplot(datasources=[
        newgrid['x'][:,0], 
        transformations.basic.STD(1, datasources=[interpolated])], 
        plottarget=plotwindow3[3,0])
p14 = mpl.lineplot(datasources=[
        newgrid['x'][:,0], 
        transformations.basic.STD(1, datasources=[interpolated2])], 
        plottarget=plotwindow3[3,1])

In [ ]:
f = h5py.File('D:/data/2017-12-29/2017-12-29_086_cq.h5', 'a')

In [ ]:
c1 = crop['Column 1']
c2 = crop['Column 2']
c4 = crop['Column 4']
c13 = crop['Column 13']
c1.play()
c2.play()
c4.play()
c13.play()
group = str(slice_nr)+"_("+str(c1.get_data()[0,0])+","+str(c2.get_data()[0,0])+")"
f[group+'/x'] = c1.get_data()
f[group+'/y'] = c2.get_data()
f[group+'/c4'] = c4.get_data()
f[group+'/c13'] = c13.get_data()

newx = newgrid['x']
newx.play()
newy = newgrid['y']
newy.play()
new4 = interpolated
new4.play()
new13 = interpolated2
new13.play()

f[group+'/newx'] = newx.get_data()
f[group+'/newy'] = newy.get_data()
f[group+'/new4'] = new4.get_data()
f[group+'/new13'] = new13.get_data()

In [ ]:
f.close()

In [ ]:
newgrid.get_data().reshape((100,100), dtype=np.dtype([('x',np.float64),('y',np.float64)]))

In [ ]:
std.play()

In [ ]:
std.get_data()

In [ ]:
t = newgrid['x'][:,0]
t.play()
t.get_data()

In [ ]:
t2 = newgrid['y'][:,0]
t2.play()
t2.get_data()

In [ ]:
t.get_data()+t2.get_data()

In [ ]:
t.get_data().dtype

In [ ]:
dt = tr1.get_data().dtype

In [ ]:
n = np.zeros((tr1.get_data().shape), dtype=dt)

In [ ]:
for name in dt.names:
    n[name]= tr1.get_data()[name]+tr1.get_data()[name]

In [ ]:
for dt in tr1.get_data().dtype:
    print(dt)

In [ ]:
len(tr1.get_data().dtype)

In [ ]:
t.get_data().dtype

In [ ]:
len(t.get_data().dtype)

In [ ]:
tr1.get_data()+tr1.get_data()

In [ ]:
t.get_data()+t.get_data()

In [ ]:
t.get_data()

In [ ]:
t.get_data()+t.get_data()

In [ ]:
import h5py

In [ ]:
f['data'] = s1.get_data()

In [ ]:
f.close()

In [ ]:
f = h5py.File('test.h5', 'a')

In [ ]:
f['group2/data'] = newgrid.get_data()
f['group2/data2'] = interpolated.get_data()

In [ ]:
f['group2/data'] = newgrid.get_data()

In [ ]:
f.get()

In [ ]:
t = f.items()

In [ ]:
for key in f.keys():
    print(key)

In [ ]:
np.array(f['phase'])

In [ ]:
t2 = np.array(f['coordinates'])

In [ ]:
t2.dtype

In [ ]:
t2.shape

In [ ]:
f.close()